## Exercise 10.44: Comparing Latin Hypercube sampling precision with Monte Carlo
This an adaptation of problem 10.44 to Python, which includes Latin Hypercube sampling capability. This is provided in a submodule of scipy.stats called qmc (quasi-monte-carlo sampling) that we import below.

For the textbook problem, there is no model building. The question asks you to run some @Risk simulations that use the two sampling methods and interpret results. We follow the same approach, i.e., provide code to generate samples and ask for interpretations through answers to the following two questions:
- What do you find?
- Why do we say that Latin Hypercube sampling is more efficient?

Answers to these questions follow from examining the results, though you are welcome to read a little about the difference between these two methods.

We include documentation within simShell and the execution code block to help clarify the inputs, simulation process, and outputs.

In brief, we want to generate 10 random samples of a normal random variable (mean = 500, standard deviation = 100). These random samples are generated according to the two sampling methods (i.e., 20 samples in total). The sample size below is set to 1000 to match the book problem, but you may experiment with different sizes if you like. 

For each random sample, we compute a number of sample statistics (sample mean, sample standard deviation, sample quantiles at 0.5 and 0.95. The idea is to see how much variation there is in these sample statics across the 10 random samples for each sampling method. Toward this end, we also compute the sample standard deviation of each sample statistic across the 10 samples. 

### Brief background on sampling & link to generating random variables in Excel
We can generate a sample of a normal random variable in Excel using two functions: 'RAND()' and 'NORM.INV(probability,mean,stdev)'. The function NORM.INV(probability,mean,stdev) maps a probability into a realization of a normal random variable with the specified mean and standard deviation). The function RAND() generates random probability, i.e., a uniformly distributed realization on \[0, 1\]. Thus, NORM.INV(RAND(),mean,stdev) maps the  probability to a realization. In Excel, we can execute NORM.INV(RAND(),mean,stdev) many times to generate a sample of a normally distributed random variable with the specified mean and standard deviation. As an aside, Excel uses Monte Carlo sampling in the execution of RAND().

Why is this background relevant? The reason is that you will see the same process used to a sample via Latin Hypercube in Python. We will use a function like RAND() to generate a sample from a uniform \[0,1\] random variable. However, for the  generation of this sample of probabilities, we use Latin Hypercube sampling (in brief, this is known as statified sampling because it draws samples in a balanced way from regions in the sample space).

After we have a sample of probabilities, we apply the inverse of the probability function (e.g., function NORM.INV(probability,mean,stdev) in Excel) to generate a sample of a normally distributed random variable. 

The default sampling method in Python is Monte Carlo, and Python has built in sampling functions (e.g., in the NumPy or SciPy libraries).

### Brief background on coding strategy
We'll generate an array where number of rows = size of each sample (e.g., 1000) and number of columns = number of samples. We can generate such an array using Latin Hypercube sampling, and another such an array using Monte Carlo sampling. The outputs from simShell will be a data frame for each sample. Then we use simStats to examine the sample statistics. 

In [1]:
# Import libraries that may be used
import numpy as np
import pandas as pd
import scipy.stats as sp
import matplotlib.pyplot as plt

## Here we import the submodule needed for Latin Hypercube sampling
from scipy.stats import qmc

In [2]:
def simStats(Z_df):
    '''Generic code to compute summary stats for each column in a data frame (input as Z_df)'''
    # Compute sample mean, sample stdev, standard error, min, max
    Mean = Z_df.mean()
    Stdev = Z_df.std(ddof=1)              # Reduce degrees of freedom (ddof) by 1 to yield sample stdev
    Quant05 = Z_df.quantile(0.05)
    Quant95 = Z_df.quantile(0.95)
    Stderr = Z_df.sem()
    Min = Z_df.min()
    Max = Z_df.max()

    # Specify probability for confidence interval of sample mean
    CI_prob = 0.95

    # Compute confidence interval
    Tail_prob = (1 - CI_prob)/2
    z = sp.norm.ppf(1-Tail_prob)
    CI_lo = Mean - z * Stderr
    CI_hi = Mean + z * Stderr
    
    # Create & return data frame of sample statistics
    Sample_stats = pd.concat([Mean,Stdev,Quant05,Quant95,Stderr,CI_lo,CI_hi,Min,Max],axis=1)
    Sample_stats.columns = ['Sample mean', 'Sample stdev', 'Quantile 0.05','Quantile 0.95',
                            'Std error','CI-lo','CI-hi','Min','Max']
    return Sample_stats

def simHistogram(Z_df):
    '''
    Generic code to plot histograms for each column in a data frame Z_df

    '''
    # Iterate over columns
    for column in Z_df.columns:
        # Clear previous plot
        plt.clf()

        # Generate histogram for current column
        plt.hist(Z_df[column], bins='auto', edgecolor='black',density=True)

        # Set labels and title
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.title(f'Histogram of {column}')
        
        # Display the histogram
        plt.show()
    
def simShell(controls,inputs):
    '''
    Generic sim code: specifies deterministic model, sets some inputs as 
    random, returns random sample of model outputs stored as a data frame
    '''
    #A Unpack the controls (standardized)
    seed = controls[0]               # Determines whether common seed is used
    n = controls[1]                  # Sample size

    #B Unpack the inputs (model-dependent)
      ## To be filled in based on model inputs

    #C Use same seed (true) or not false) for random number streams
    if seed:
        np.random.seed(100)          # Can change the seed value
    
    #D Generate sample(s) of random inputs & outputs (eg, in arrays)
      ## Code for generating random inputs from probability distributions
      ## Code that maps inputs to outputs (i.e., the model)
    
    #E Store array of outputs (& desired inputs) in data frame & return
    Z_df = pd.DataFrame(Z)
    Z_df.columns = ['label names as approriate']
    return Z_df   

### Code for the problem

In [3]:
def simShell(controls,inputs):
    '''
    Generic sim code: specifies deterministic model, sets some inputs as 
    random, returns random sample of model outputs stored as a data frame
    '''
    #A Unpack the controls (standardized)
    seed = controls[0]               # Determines whether common seed is used
    n = controls[1]                  # Sample size

    #B Unpack the inputs (model-dependent)
    N = inputs[0]
    mu = inputs[1]
    stdev = inputs[2]
     
    #C Use same seed (true) or not false) for random number streams
    if seed:
        np.random.seed(100)          # Can change the seed value
    
    #D Generate sample(s) of random inputs & outputs (eg, in arrays)
    ## Latin Hypercube (LH) sampling
    sampler = qmc.LatinHypercube(d=N)                   # Create an LH sampler
    probs = sampler.random(n=n)                         # Generate the LH samples (uniform between 0 and 1)
    Z1 = sp.norm.ppf(probs,mu,stdev)                    # Map sample of probabilities to sample of normals
    ## Monte Carlo (MC) sampling
    Z2 = sp.norm.rvs(loc=mu,scale=stdev,size=(n,N))     # Generate MC sample (alt, np.normal.random('same'))
    
    #E Store array of outputs (& desired inputs) in data frame & return
    Z1_df = pd.DataFrame(Z1)
    Z2_df = pd.DataFrame(Z2)
    return Z1_df,Z2_df

In [4]:
# Simulation control parameters 
## common seed indicator, sample size
controls = [False,1000]

# Model inputs: number of samples, mean, standard deviation
inputs = [10,500,100]

# Run Monte Carlo simulation and display results
Z1_df,Z2_df = simShell(controls,inputs)      # Generate random samples
LH_df=simStats(Z1_df)
MC_df=simStats(Z2_df)
print('Sample statistics for the Latin Hypercube samples')
display(LH_df)
print('Sample statistics for the Monte Carlo samples')
display(MC_df)

Sample statistics for the Latin Hypercube samples


,Sample mean,Sample stdev,Quantile 0.05,Quantile 0.95,Std error,CI-lo,CI-hi,Min,Max
0,500.025587,99.978344,336.020934,663.879335,3.161593,493.828979,506.222195,180.830418,820.874253
1,499.998608,99.930760,336.073042,663.638290,3.160088,493.804949,506.192267,183.049386,827.413186
2,500.021858,99.978459,335.952649,664.501001,3.161596,493.825243,506.218473,184.370768,831.928213
3,500.018454,99.940456,335.956129,664.192052,3.160395,493.824194,506.212713,190.462312,836.419840
4,499.982264,99.934552,335.813167,663.966935,3.160208,493.788370,506.176158,178.266901,819.028661
5,499.978843,100.132156,336.119851,664.422286,3.166457,493.772702,506.184984,140.229924,842.642952
6,500.003822,99.987300,335.541143,664.343277,3.161876,493.806659,506.200985,170.949270,825.911761
7,500.013278,100.382936,335.662028,664.279087,3.174387,493.791593,506.234962,124.799169,894.446358
8,500.009366,99.958685,335.739100,663.731630,3.160971,493.813977,506.204756,185.302990,824.070619
9,499.985711,99.946724,335.580470,663.930897,3.160593,493.791062,506.180359,168.852594,820.088658


Sample statistics for the Monte Carlo samples


,Sample mean,Sample stdev,Quantile 0.05,Quantile 0.95,Std error,CI-lo,CI-hi,Min,Max
0,498.813420,98.154681,333.530001,653.965617,3.103924,492.729841,504.896998,154.277382,776.298020
1,499.145867,102.438354,332.288278,667.497010,3.239385,492.796788,505.494945,169.549367,851.430505
2,495.930337,103.604706,321.671615,666.241796,3.276268,489.508969,502.351705,125.403997,863.416241
3,500.312405,101.694178,341.215043,675.883904,3.215852,494.009450,506.615359,179.817842,825.469971
4,503.833961,99.117668,343.555336,664.107568,3.134376,497.690697,509.977225,186.423741,786.848182
5,502.205653,98.495078,343.589640,663.790184,3.114688,496.100977,508.310329,166.019172,797.846008
6,495.318890,98.617174,336.297759,657.407862,3.118549,489.206647,501.431133,187.584391,861.478254
7,498.838299,101.757045,330.152826,669.653135,3.217840,492.531448,505.145150,181.873286,796.841039
8,499.314002,96.773208,343.614067,659.685096,3.060238,493.316047,505.311958,190.928444,817.472841
9,503.384557,95.960273,347.807934,656.221900,3.034530,497.436987,509.332127,204.636079,871.034738


In [5]:
print('Summary statistics of the sample means and sample standard deviations for Latin Hypercube samples')
display(simStats(LH_df[['Sample mean','Sample stdev']]))
print('Summary statistics of the sample means and sample standard deviations for Monte Carlo samples')
display(simStats(MC_df[['Sample mean','Sample stdev']]))

Summary statistics of the sample means and sample standard deviations for Latin Hypercube samples


,Sample mean,Sample stdev,Quantile 0.05,Quantile 0.95,Std error,CI-lo,CI-hi,Min,Max
Sample mean,500.003779,0.016922,499.980382,500.023909,0.005351,499.993291,500.014267,499.978843,500.025587
Sample stdev,100.017037,0.141211,99.932466,100.270085,0.044655,99.929515,100.104559,99.930760,100.382936


Summary statistics of the sample means and sample standard deviations for Monte Carlo samples


,Sample mean,Sample stdev,Quantile 0.05,Quantile 0.95,Std error,CI-lo,CI-hi,Min,Max
Sample mean,499.709739,2.842282,495.594041,503.631729,0.898808,497.948107,501.471371,495.318890,503.833961
Sample stdev,99.661236,2.556943,96.326094,103.079847,0.808576,98.076456,101.246017,95.960273,103.604706


### Explain your interpretations of the results from the sampling methods here: 

#### What do you find? 

#### Why do we say that Latin Hypercube is more efficient?



## Observations : 

LHS: Sample means are tightly clustered around 500 (mean = 500.004, stdev = 0.017). Standard deviations are also stable (mean = 100.017, stdev = 0.141).
LHS: Narrow CIs (e.g., 493.8–506.2) reflect high precision.
LHS: Min/max values (e.g., 124.8–894.4) span a large range but do not destabilize estimates.

MC: Sample means vary significantly (mean = 499.71, stdev = 2.842), and standard deviations fluctuate more (mean = 99.66, stdev = 2.557).
MC: Wider CIs (e.g., 489.2–509.3) indicate lower precision.
MC: Greater variability in extremes (e.g., 125.4–871.0) contributes to less reliable estimates.

## Conclusion:
Latin Hypercube produces more consistent, precise, and reliable results with less variability, and hence it is a more efficient method for complex simulations.
